In [38]:
#First, the point cloud has to be loaded and then read.
#Its probably way too big to work with from the start, so I think I will use voxel downsampling
#But first, imports

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os   


#Loading the point cloud

initialCloud =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "croppedRawData.ply")) 




In [40]:
#Making a voxelgrid from the point cloud
voxel_size = 1
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(initialCloud, voxel_size)



In [41]:
o3d.visualization.draw_geometries([voxel_grid])

In [42]:
#Extracting center points so we can calculate PCA for the points within each voxel

voxel_center_coordinates= []

for i in voxel_grid.get_voxels():
    voxel_center_coordinates.append(voxel_grid.get_voxel_center_coordinate(i.grid_index))



In [43]:
#Defining a function to extract points from a voxel based on center coordinates. Returns a list of points

from sklearn.decomposition import PCA
from tqdm.auto import tqdm


def extractPointCloudFromVoxel(voxel_center_coordinates, pointCloud, voxel_size):
    extractedPointClouds = []
    for center in tqdm(voxel_center_coordinates):
        minboundx = center[0] - voxel_size/2
        minboundy = center[1] - voxel_size/2
        minboundz = center[2] - voxel_size/2

        maxboundx = center[0] + voxel_size/2
        maxboundy = center[1] + voxel_size/2
        maxboundz = center[2] + voxel_size/2

        pcd = pointCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
        # Voxels with less than 10 points are ignored and not appended
        
        if len(pcd.points) > 10:
            extractedPointClouds.append(pcd)
    return extractedPointClouds
    
    
    

In [44]:
#Function to calculate features from a list of points clouds (could be one or more), returns lists for each feature
def calculatePCAFeatures(listOfPointClouds):
    
    linearity =[]
    planarity = []
    scattering = []
    omnivariance = []
    sum_eigenvalues = []
    anisotropy = []
    eigentropy = []
    change_of_curvature = []
    eigenvaluesList = []
    eigenvectorsList = []
    points_per_voxel = []
    
    for pcd in tqdm(listOfPointClouds):
        points = pcd.points
    
        '''
        if len(points) < 15:
            linearity.append(0)
            planarity.append(0)
            scattering.append(0)
            omnivariance.append(0)
            sum_eigenvalues.append(0)
            anisotropy.append(0)
            eigentropy.append(0)
            change_of_curvature.append(0)
            points_per_voxel.append(points) #Kan her appende 0 istedenfor points?
            continue
            ''' #Denne sjekken blir gjort i extractPointCloudFromVoxel, slik at man slipper å gjøre det her
    
        pca = PCA(n_components=3)
        pca.fit(points)

    # Extract the eigenvalues and eigenvectors
        eigenvalues = pca.explained_variance_
        eigenvectors = pca.components_
    
      #Calculating the features
        l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
        p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
        s = eigenvalues[2] / eigenvalues[0]
        o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
        s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
        a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
        e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
        c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])

        linearity.append(l)
        planarity.append(p)
        scattering.append(s)
        omnivariance.append(o)
        sum_eigenvalues.append(s_e)
        anisotropy.append(a)
        eigentropy.append(e)
        change_of_curvature.append(c_o_c)
        eigenvaluesList.append(eigenvalues)
        eigenvectorsList.append(eigenvectors)
        points_per_voxel.append(points)
        
    return linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, eigenvaluesList, eigenvectorsList, points_per_voxel

In [46]:
#Defining a seperate function to calculate z-range for a set of points
def calculateZRange(pointClouds):
    z_range = []
    for pcd in tqdm(pointClouds):
        points = np.asarray(pcd.points)
        '''
        if len(points) < 15:
            z_range.append(0)
            continue
            ''' #Denne sjekken blir gjort i extractPointCloudFromVoxel
        z = np.max(points[:,2]) - np.min(points[:,2])
        z_range.append(z)
    return z_range
   

In [47]:
#Extracting the points from each voxel

extractedPointClouds = extractPointCloudFromVoxel(voxel_center_coordinates, initialCloud, voxel_size)

  0%|          | 0/450800 [00:00<?, ?it/s]

In [48]:
#Calculating the features for each voxel
linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, eigenvalues, eigenvectors, points_per_voxel = calculatePCAFeatures(extractedPointClouds)

  0%|          | 0/242778 [00:00<?, ?it/s]

In [49]:
#Calculating z-range for each voxel IT WORKS

z_range = calculateZRange(extractedPointClouds)


  0%|          | 0/242778 [00:00<?, ?it/s]

In [114]:
#Making a dataframe with all the features and the points for each voxel with an empty label column

import pandas as pd

# both lists, with columns specified
# label = [0] * len(linearity) #Placeholder zeroes

df = pd.DataFrame(list(zip(points_per_voxel, linearity, planarity, scattering, omnivariance, sum_eigenvalues, anisotropy, eigentropy, change_of_curvature, z_range, )),
			columns =[ 'points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'z_range', ])



In [135]:
df

,points,linearity,planarity,scattering,omnivariance,sum_eigenvalues,anisotropy,eigentropy,change_of_curvature,z_range,label
0,"[[569901.039978, 7034055.560059, 22.26], [5699...",0.504846,0.395950,0.099204,0.030882,0.134440,0.900796,0.381160,0.062222,0.690000,1.0
1,"[[570046.660034, 7034121.640015, 8.44], [57004...",0.286875,0.711114,0.002011,0.013096,0.199191,0.997989,0.458306,0.001172,0.060000,1.0
2,"[[569918.659973, 7034220.420044, 28.35], [5699...",0.400117,0.498582,0.101301,0.028418,0.122968,0.898699,0.361980,0.059547,0.440000,1.0
3,"[[569998.049988, 7034287.170044, 3.73], [56999...",0.950197,0.023885,0.025918,0.008723,0.086185,0.974082,0.237107,0.024093,0.210000,0.0
4,"[[569957.640015, 7034200.109985, 18.48], [5699...",0.288012,0.708027,0.003960,0.016018,0.194557,0.996040,0.453456,0.002308,0.489999,1.0
...,...,...,...,...,...,...,...,...,...,...,...
242773,"[[570142.150024, 7034182.359985, 26.65], [5701...",0.646421,0.351680,0.001899,0.008801,0.136231,0.998101,0.351140,0.001401,0.119999,2.0
242774,"[[570170.660034, 7034180.209961, 25.559999], [...",0.457261,0.537273,0.005467,0.013555,0.146043,0.994533,0.378774,0.003531,0.160000,1.0
242775,"[[570084.650024, 7034302.060059, 2.98], [57008...",0.504146,0.495516,0.000338,0.006641,0.180316,0.999662,0.423787,0.000226,0.020000,2.0
242776,"[[570177.699951, 7034300.01001, 7.61], [570177...",0.159510,0.839737,0.000753,0.008556,0.183453,0.999247,0.438179,0.000409,0.080000,2.0


In [52]:
df.to_csv(os.path.join(os.getcwd(), "data", "calculatedFeaturesRawData.csv")) #Saving the dataframe to a csv file

In [14]:
import os
#Read all the houses into a list

houses_folder = os.path.join(os.getcwd(), "data", "Houses")
houses_files = os.listdir(houses_folder)

houses = []
for file in houses_files:
    if file.endswith(".ply"):
        house = o3d.io.read_point_cloud(os.path.join(houses_folder, file))
        houses.append(house)


In [15]:
#Load all the trees into a list

trees_folder = os.path.join(os.getcwd(), "data", "Trees")   
trees_files = os.listdir(trees_folder)

trees = []
for file in trees_files:
    if file.endswith(".ply"):
        tree = o3d.io.read_point_cloud(os.path.join(trees_folder, file))
        trees.append(tree)


In [16]:
#Load all terrain into a list

terrain_folder = os.path.join(os.getcwd(), "data", "Terrain")
terrain_files = os.listdir(terrain_folder)

terrain = []
for file in terrain_files:
    if file.endswith(".ply"):
        terrain_piece = o3d.io.read_point_cloud(os.path.join(terrain_folder, file))
        terrain.append(terrain_piece)


In [ ]:
#Parameters for houses

#First making a point cloud of all houses
houseCloud = o3d.geometry.PointCloud()
for house in houses:
    houseCloud+= house
    
#Making a voxel grid of the houses
houseVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(houseCloud, voxel_size)

#Extracting center points
houseVoxelCenterCoordinates= []
for i in houseVoxelGrid.get_voxels():
    houseVoxelCenterCoordinates.append(houseVoxelGrid.get_voxel_center_coordinate(i.grid_index))
    
#Extracting points from each voxel
houseExtractedPointClouds = extractPointCloudFromVoxel(houseVoxelCenterCoordinates, houseCloud, voxel_size)

#Calculating the PCA features for each voxel
houseLinearity, housePlanarity, houseScattering, houseOmnivariance, houseSum_eigenvalues, houseAnisotropy, houseEigentropy, houseChange_of_curvature, houseEigenvalues, houseEigenvectors, housePoints_per_voxel = calculatePCAFeatures(houseExtractedPointClouds)

#Calculating other features
houseZ_range = calculateZRange(houseExtractedPointClouds)

#Making an array of labels for the houses

houseLabels = np.ones(len(houseLinearity))

#Making a dataframe with all the features and the points for each voxel
dfHouse = pd.DataFrame(list(zip(housePoints_per_voxel, houseLinearity, housePlanarity, houseScattering, houseOmnivariance, houseSum_eigenvalues, houseAnisotropy, houseEigentropy, houseChange_of_curvature, houseZ_range, houseLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'z_range', 'label'])

In [ ]:
#Parameters for trees

#First making a point cloud of all trees
treeCloud = o3d.geometry.PointCloud()
for tree in trees:
    treeCloud+= tree
    
#Making a voxel grid of the trees
treeVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(treeCloud, voxel_size)

#Extracting center points
treeVoxelCenterCoordinates= []
for i in treeVoxelGrid.get_voxels():
    treeVoxelCenterCoordinates.append(treeVoxelGrid.get_voxel_center_coordinate(i.grid_index))

#Extracting points from each voxel
treeExtractedPointClouds = extractPointCloudFromVoxel(treeVoxelCenterCoordinates, treeCloud, voxel_size)

#Calculating the PCA features for each voxel
treeLinearity, treePlanarity, treeScattering, treeOmnivariance, treeSum_eigenvalues, treeAnisotropy, treeEigentropy, treeChange_of_curvature, treeEigenvalues, treeEigenvectors, treePoints_per_voxel = calculatePCAFeatures(treeExtractedPointClouds)

#Calculating other features
treeZ_range = calculateZRange(treeExtractedPointClouds)

#Making an array of labels for the trees
treeLabels = np.zeros(len(treeLinearity))   

#Making a dataframe with all the features and the points for each voxel
dfTree = pd.DataFrame(list(zip(treePoints_per_voxel, treeLinearity, treePlanarity, treeScattering, treeOmnivariance, treeSum_eigenvalues, treeAnisotropy, treeEigentropy, treeChange_of_curvature, treeZ_range, treeLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'z_range', 'label'])



In [ ]:
#Parameters for terrain

#First making a point cloud of all terrain
terrainCloud = o3d.geometry.PointCloud()

for terrain_piece in terrain:
    terrainCloud+= terrain_piece

#Making a voxel grid of the terrain
terrainVoxelGrid = o3d.geometry.VoxelGrid.create_from_point_cloud(terrainCloud, voxel_size)

#Extracting center points
terrainVoxelCenterCoordinates= []
for i in terrainVoxelGrid.get_voxels():
    terrainVoxelCenterCoordinates.append(terrainVoxelGrid.get_voxel_center_coordinate(i.grid_index))

#Extracting points from each voxel
terrainExtractedPointClouds = extractPointCloudFromVoxel(terrainVoxelCenterCoordinates, terrainCloud, voxel_size)

#Calculating the PCA features for each voxel
terrainLinearity, terrainPlanarity, terrainScattering, terrainOmnivariance, terrainSum_eigenvalues, terrainAnisotropy, terrainEigentropy, terrainChange_of_curvature, terrainEigenvalues, terrainEigenvectors, terrainPoints_per_voxel = calculatePCAFeatures(terrainExtractedPointClouds)

#Calculating other features
terrainZ_range = calculateZRange(terrainExtractedPointClouds)

#Making an array of labels for the terrain with number 2
terrainLabels = np.full(len(terrainLinearity), 2)

#Making a dataframe with all the features and the points for each voxel
dfTerrain = pd.DataFrame(list(zip(terrainPoints_per_voxel, terrainLinearity, terrainPlanarity, terrainScattering, terrainOmnivariance, terrainSum_eigenvalues, terrainAnisotropy, terrainEigentropy, terrainChange_of_curvature, terrainZ_range, terrainLabels)),
                       columns = ['points', 'linearity', 'planarity', 'scattering', 'omnivariance', 'sum_eigenvalues', 'anisotropy', 'eigentropy', 'change_of_curvature', 'z_range', 'label'])




In [ ]:
#Now its time for the machine learning part

#First, we need to split the data into training and testing data, and also try to drop some features to see if it improves the accuracy

from sklearn.model_selection import train_test_split

#Combining the datasets into one dataframe
dfCombined = pd.concat([dfHouse, dfTree, dfTerrain], ignore_index=True)


#Placing the labels in a dataframe
y_train = dfCombined['label']

#Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfCombined.drop(['points', 'label',], axis=1), dfCombined['label'], test_size=0.20, random_state=42)




In [119]:
#Reading the training data from the csv file

dfTraining = pd.read_csv(os.path.join(os.getcwd(), "data", "voxelFeatures.csv"))

In [120]:
dfTraining.drop(['Unnamed: 0'], axis=1, inplace=True)

In [122]:
#Placing the labels in a dataframe
y_train = dfTraining['label']

#Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfTraining.drop(['points', 'label',], axis=1), dfTraining['label'], test_size=0.20, random_state=42)

In [127]:
#Training a decision tree classifier

from sklearn.tree import DecisionTreeClassifier


#Creating the classifier
clf = DecisionTreeClassifier(random_state=0)

#Training the classifier
clf.fit(X_train, y_train)

#Testing it
y_pred = clf.predict(X_test)



In [ ]:
# Initialize a CatBoost model with Decision Tree as the base learner
from catboost import CatBoostClassifier


catboost_model = CatBoostClassifier(
    iterations=100,  # Adjust as needed
    depth=6,          # Adjust as needed
    learning_rate=0.1,
    custom_metric=['Accuracy'],
    loss_function='Logloss',
    random_seed=42
)



In [ ]:
#Testing the accuracy of the classifier

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

feature_importances = clf.feature_importances_
print(feature_importances)



In [129]:
#Now actually testing it on the point cloud

#We already have the dataframe for the entire point cloud, so we can just use that

X_cloud = df.drop(['points'], axis=1)

#Predicting the labels for the point cloud
y_cloud_pred = clf.predict(X_cloud)

#Adding the labels to the dataframe
df['label'] = y_cloud_pred

In [130]:
df

,points,linearity,planarity,scattering,omnivariance,sum_eigenvalues,anisotropy,eigentropy,change_of_curvature,z_range,label
0,"[[569901.039978, 7034055.560059, 22.26], [5699...",0.504846,0.395950,0.099204,0.030882,0.134440,0.900796,0.381160,0.062222,0.690000,1.0
1,"[[570046.660034, 7034121.640015, 8.44], [57004...",0.286875,0.711114,0.002011,0.013096,0.199191,0.997989,0.458306,0.001172,0.060000,1.0
2,"[[569918.659973, 7034220.420044, 28.35], [5699...",0.400117,0.498582,0.101301,0.028418,0.122968,0.898699,0.361980,0.059547,0.440000,1.0
3,"[[569998.049988, 7034287.170044, 3.73], [56999...",0.950197,0.023885,0.025918,0.008723,0.086185,0.974082,0.237107,0.024093,0.210000,0.0
4,"[[569957.640015, 7034200.109985, 18.48], [5699...",0.288012,0.708027,0.003960,0.016018,0.194557,0.996040,0.453456,0.002308,0.489999,1.0
...,...,...,...,...,...,...,...,...,...,...,...
242773,"[[570142.150024, 7034182.359985, 26.65], [5701...",0.646421,0.351680,0.001899,0.008801,0.136231,0.998101,0.351140,0.001401,0.119999,2.0
242774,"[[570170.660034, 7034180.209961, 25.559999], [...",0.457261,0.537273,0.005467,0.013555,0.146043,0.994533,0.378774,0.003531,0.160000,1.0
242775,"[[570084.650024, 7034302.060059, 2.98], [57008...",0.504146,0.495516,0.000338,0.006641,0.180316,0.999662,0.423787,0.000226,0.020000,2.0
242776,"[[570177.699951, 7034300.01001, 7.61], [570177...",0.159510,0.839737,0.000753,0.008556,0.183453,0.999247,0.438179,0.000409,0.080000,2.0


In [131]:
#Now we need to extract the points from the trees and houses

#Making a dataframe for trees and houses
dfTrees = df[df['label'] == 0]
dfHouses = df[df['label'] == 1]
dfTerrain = df[df['label'] == 2]

#Extracting the point-clusters from the trees
treeClusters= dfTrees['points']
houseClusters = dfHouses['points']
terrainClusters = dfTerrain['points']

classifiedTreeCloud = o3d.geometry.PointCloud()
classifiedHouseCloud = o3d.geometry.PointCloud()
classifiedTerrainCloud = o3d.geometry.PointCloud()

#Painting them

#Trees get green
for cluster in treeClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([0, 1, 0])  
    classifiedTreeCloud += cld

#Houses get red
for cluster in houseClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([1, 0, 0])  
    classifiedHouseCloud += cld
    
#Terrain gets yellow
for cluster in terrainClusters:
    cld = o3d.geometry.PointCloud(cluster)
    cld.paint_uniform_color([0, 0, 1])
    classifiedTerrainCloud += cld


In [134]:
o3d.visualization.draw_geometries([classifiedTreeCloud, classifiedHouseCloud, classifiedTerrainCloud])